In [ ]:
## 4.1 브랜드별 전체 주문건, 총 매출

# 브랜드별 2022년도와 2022년도 이하 모든 연도의 전체주문건수, 판매금액합계, 순위를 조회하세요.

# **조회 항목**

# - 브랜드명 (brand)
# - 모든 연도 주문 건수 (order_count_total)
# - 2022년도 주문 건수 (order_count_2022)
# - 모든 연도 판매금액 합계 (sum_sale_price_total)
# - 2022년도 판매금액 합계 (sum_sale_price_2022)
# - 2022년도 판매금액 합계 순위 (rank_by_sum_sale_2022)
# - 2022년도 주문건수 순위 (rank_by_order_count_2022)

# **정렬 순서**

# - 2022년도 판매금액 합계 순위(rank_by_sum_sale_2022) 오름차순

WITH brand_sum_price AS
(
  SELECT
    brand,
    COUNT(DISTINCT t1.order_id) AS order_count_total,
    COUNT(DISTINCT (CASE WHEN t1.created_at >= '2022-01-01'
    AND t1.created_at < '2023-01-01' THEN t1.order_id END)) AS order_count_2022,
    ROUND(SUM(t1.sale_price),2) AS sum_sale_price_total,
    ROUND(SUM(CASE WHEN t1.created_at >= '2022-01-01'
    AND t1.created_at < '2023-01-01' THEN t1.order_id END), 2) AS sum_sale_price_2022
  FROM `thelook_ecommerce.order_items` t1
  LEFT JOIN `thelook_ecommerce.products` t2
  ON t1.product_id = t2.id
  LEFT JOIN `thelook_ecommerce.users` t3
  ON t1.user_id = t3.id
  WHERE t1.created_at < '2023-01-01'
  GROUP BY brand
)
SELECT
  *,
  RANK() OVER(ORDER BY sum_sale_price_2022 DESC) AS rank_by_sum_sale_2022,
  RANK() OVER(ORDER BY order_count_2022 DESC) AS rank_by_order_count_2022
  FROM brand_sum_price
  ORDER BY rank_by_sum_sale_2022

In [ ]:
### 4.2.1 **매출이 낮은 브랜드 확인 (상품개수 약 10개 이상, 매출 하위 60%)**

# 매출하위 60%, 상품개수 10개 이상의 브랜드의 정보를 조회하세요.

# **조회 항목**

# - 브랜드명(brand)
# - 상품수(product_count)
# - 판매금액 합계 (sum_sale_price
# - 판매금액기준 퍼센트랭크 (percent_rank_sum_sale_price)

# **정렬 순서**

# - 판매금액기준 퍼센트랭크 (percent_rank_sum_sale_price) 오름차순

WITH brand_over_10 AS (
  SELECT
    brand,
    COUNT(id) AS product_count
  FROM `thelook_ecommerce.products`
  WHERE brand IS NOT NULL
  GROUP BY brand
  HAVING product_count >= 10
),
sum_price_2022 AS (
  SELECT
    t2. brand,
    ROUND(SUM(t1.sale_price),2) AS sum_sale_price
  FROM `thelook_ecommerce.order_items` t1
  LEFT JOIN `thelook_ecommerce.products` t2
  ON t1.product_id = t2.id
  LEFT JOIN `thelook_ecommerce.users` t3
  ON t1.user_id = t3.id
  WHERE t1.created_at >= '2022-01-01'
  AND t1.created_at < '2023-01-01'
  GROUP BY brand
),
brand_rank_list AS (
  SELECT
    *,
    PERCENT_RANK() OVER(ORDER BY a2.sum_sale_price) AS percent_rank_sum_sale_price
  FROM brand_over_10 a1
  LEFT OUTER JOIN sum_price_2022 a2
    ON a1.brand = a2.brand
)
SELECT *
FROM brand_rank_list
WHERE percent_rank_sum_sale_price <= 0.6
ORDER BY percent_rank_sum_sale_price

In [ ]:
# 위와 같은 코드.

WITH brand_ranks AS (
  SELECT
    COUNT(DISTINCT t1.product_id) AS product_count,
    ROUND(SUM(t1.sale_price), 2) AS sum_sale_price,
    PERCENT_RANK() OVER(
      ORDER BY SUM(t1.sale_price)
    ) AS percent_rank_sum_sale_price
  FROM `thelook_ecommerce.order_items` t1
  JOIN `thelook_ecommerce.products` t2
  ON t1.product_id = t2.id
  WHERE t2.brand IS NOT NULL
  AND t1.created_at >= '2022-01-01'
  AND t1.created_at < '2023-01-01'
  GROUP BY t2.brand
  HAVING product_count >= 10
)
SELECT *
FROM brand_ranks
WHERE percent_rank_sum_sale_price <= 0.6

In [ ]:
## 4.3 상품 선정 (1개 상품)

# 여기서는 최근 3개월 매출이 가장 높은 `Rufskin` 브랜드로 분석하도록 하겠습니다.
# 실무에서는 6개 광고 대상 브랜드를 모두 분석해야 합니다.

# ### 4.3.1 30대 이하 유저의 매출순위가 높은 상품

# `Rufskin` 브랜드의 30대이하 유저 판매기록이 높은 10개 상품을 조회하세요.

# **조회 항목**

# - 상품ID (product_id)
# - 상품명 (name)
# - 30대이하 유저 판매기록 합계 (sum_sale_price)

# **정렬 순서**

# - 30대이하 유저 판매기록 합계 (sum_sale_price)

SELECT
  t1.product_id,
  t3.name,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price
FROM `thelook_ecommerce.order_items` t1
LEFT JOIN `thelook_ecommerce.users` t2
ON t1.user_id = t2.id
LEFT JOIN `thelook_ecommerce.products` t3
ON t1.product_id = t3.id
WHERE t2.age < 40
AND t3.brand = 'Rufskin'
GROUP BY 1, 2
ORDER BY 3 DESC
limit 10

In [ ]:
### 4.3.2 선정된 상품이 어떤 성별, 지역에서 인기가 있는지 추적 (TOP 1 성별, TOP 1 지역)

# 1) **4.3.1**에서 가장 매출이 높았던 제품의 지역별 판매금액을 조회해보세요.

# **조회 항목**

# - 국가명(country)
# - 판매금액합계(sum_sale_price)

# **정렬 순서**

# - 판매금액합계(sum_sale_price) 내림차순

SELECT
  t2.gender,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price
FROM `thelook_ecommerce.order_items` t1
LEFT JOIN `thelook_ecommerce.users` t2
ON t1.user_id = t2.id
WHERE product_id = 21466
GROUP BY 1
ORDER BY 2 DESC

In [ ]:
### 4.3.3 어떤 요일에 광고를 집행하면 좋을지

# **4.3.1**에서 가장 매출이 높았던 제품의 요일별 판매금액을 조회해보세요.

# **조회 항목**

# - 요일(day_of_week) : `format_date('%A', t1.created_at)`
# - 판매금액합계(sum_sale_price)

# **정렬 순서**

# - 판매금액합계(sum_sale_price) 내림차순

SELECT
  FORMAT_DATE('%A', t1.created_at) AS day_of_week,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price
FROM `thelook_ecommerce.order_items` t1
LEFT JOIN `thelook_ecommerce.users` t2
ON t1.user_id = t2.id
WHERE product_id = 21466
GROUP BY 1
ORDER BY 2 DESC

In [ ]:
# 4.4 최종 선정

# - 6개의 광고 대상 브랜드와 대표 상품

# - 함께 광고할 추천 상품(유사 상품, 같이 표시할 상품)

#     21466 상품 중 함께 광고할 추천상품은 해당 상품과 카테고리가 비슷한 상품 중
#     다음으로 많이 주문된 제품 5개로 선정하겠습니다.

#     21466 상품과 같은 카테고리, 같은 브랜드 제품중에서 매출금액이 높은 5개 제품 정보를 조회하세요.

#     **조회 항목**

#     - 상품 ID (product_id)
#     - 상품명 (name)
#     - 상품 카테고리 (category)
#     - 판매금액 합계 (sum_sale_price)
#     - 최근 주문 일시 (last_order_datetime)

#     **정렬 순서**

#     - 판매금액 합계 (sum_sale_price) 내림차순

SELECT
  t1.product_id,
  t2.name,
  t2.category,
  ROUND(SUM(t1.sale_price), 2) AS sum_sale_price,
  MAX(t1.sale_price) AS last_order_datetime
FROM `thelook_ecommerce.order_items` t1
LEFT JOIN `thelook_ecommerce.products` t2
ON t1.product_id = t2.id
WHERE t2.category = (
  SELECT category
  FROM `thelook_ecommerce.products`
  WHERE id = 21466
)
AND t2.brand =(
  SELECT brand
  FROM `thelook_ecommerce.products`
  WHERE id = 21466
)
AND product_id != 21466
GROUP BY 1, 2, 3
ORDER BY 4 DESC
LIMIT 5

In [ ]:
# - 광고대상 user_id 목록

#     광고대상을 남성이고, 30대 이하인 주 이용고객으로 목록을 정할 수 있습니다.
#     국가 = 미국,브라질,영국
#     **조회 항목**

#     - 유저 ID(id)
#     - 이름(name), first_name last_name을 이용
#     - 이메일(email)
#     - 나이(age)
#     - 성별(gender)
#     - 국가(country)

SELECT
  id,
  CONCAT(first_name, ' ', last_name) AS name,
  email,
  age,
  gender,
  country
FROM `thelook_ecommerce.users`
WHERE age < 40
AND gender = 'M'
AND country in ('United States', 'Brasil', 'United Kingdom')